In [83]:
import os
import pandas as pd
import re
import numpy as np

def parse_annotation(annot):
    list_eval_words = []
    #print ("Annot: ", str(annot[0]))
    without_spaces = str(annot[0]).replace(" ", "")
    e = re.search('/:.*/:', without_spaces)
    occurrences = without_spaces.split(";")
    #print ("Occurrences: ", occurrences)

    for occurrence in occurrences:   
        print ("OCCURENCE DEBUG: ", occurrence) #
        #####Following are malformed strings, need to change formatting
        invalid_strings = ['入口:難搵:-唔洗等位:+','餸菜:有驚喜+/餐單:好想試','皇后街熟食市場:真係幾好多野:+','炒蛋:好香+/好滑+/厚多士:勁正+','白酒蜆意粉:茨汁多左-/煙鴨胸炒飯:不過不失0/蟹肉:好多蟹肉蟹籽+/口感唔錯+','魚肉:更為嫩滑+/酒香淡淡+/朱古力:幾濃+','魚肉:嫩滑+/酒香:淡淡而發都值得加分+/朱古力:幾濃郁+/好飲+','蟹肉天使麵:好食+/火焰豬手:酥脆脆+/有質感+','叉燒包:賣相贏晒:+','蜜糖味雞蛋仔:蜜糖味偏甜搶咗蛋香-/橙朱古力雞蛋仔:唔錯+','雞柳炒飯底:好出色+/地道凍檸茶:絕配+','公仔面:夠彈+湯:鮮味+,陣陣薑味0咖央西多:新鮮+,夠熱+','中辣好辣-/米線:麻麻地-/湯底:麻麻地-/凍檸茶:太多冰','中辣好辣-/米線同湯底:麻麻地-/凍檸茶:太多冰-','濃濃湯底:正+/米線:好滑+/唔會太諗-/雲吞皮:厚咗少少-/味道都唔錯+','湯底:唔夠濃-/蝦:雪到太耐-/質地完全被破壞-/無哂應有口感-/秀珍菇:同成碗味道格格不入-', '咖啡:很濃+/洒:很香+','咖啡味:很濃+/洒味:很香+','法式焗田螺薯蓉:好食+/煙三文魚沙律:好味個汁好夾+','印度雞肉炒飯:雞肉:好好味+','韮菜:唔夠香-/韮菜粿:有少少寡-/唔夠豬油味-', '小籠包:皮薄餡靚+/素菜筍尖炒年糕:味道濃+', '鵝.蚊.件豆腐.蚊.隻蛋.共.蚊:好鹹-/豆腐:超鹹-', '饀:好足料+/餃子皮:唔會太厚+', '牛腩:好林+/好味+湯底:好飲:+河粉:好滑:+']
        #####
        if ":" in occurrence:
            if any(x in occurrence for x in invalid_strings):             
                break
            else:
                (aspect_word, eval_word_s) = occurrence.split(":")  
                #print ("Aspect_word: ", aspect_word, " eval word: ", eval_word_s)
                splitted_eval_words = process_eval_words(eval_word_s)
                list_eval_words.extend(splitted_eval_words)
        else:
            pass#print ("eval_word without aspect: ", occurrence)
            #list_eval_words.extend(occurrence)
    #print ("LIST_OF_EVAL_WORDS", list_eval_words)
        
    return (list_eval_words)


def process_eval_words(eval_word_s): # input: 十分酥嫩+/重要冇膻味+/好多肉汁+/幾好+ output: list
    # some got it wrong: 豐富+,清新+,新鮮+,濃+,甜+
    if "," in eval_word_s:
        list_of_eval_words = eval_word_s.split(",")
    else: # hoping all the rest did it correctly
        list_of_eval_words = eval_word_s.split("/")
    
    return list_of_eval_words

def compare_annotations(annot1, annot2): # now without comp-arison, just merge. TODO comparison itself!!!!
    parsed_1 = parse_annotation(annot1)
    parsed_2 = parse_annotation(annot2)
    #print ("PARSED1: ", parsed_1, "||||||| Parsed2: ", parsed_2 )
    return (parsed_1, parsed_2)
    #print ("Comparing ", without_spaces1, " and ", without_spaces2 )
    #if without_spaces1 == without_spaces2:        
    #    pass#print ("The full match!: ", without_spaces1)

def parse_frame(all_frame, columns_TODO): # After we will select not only 'taste words..'
    columns = ['taste words'] 
    words = []
    for group in all_frame.groupby('id'):
        #print ( "group[1].loc[:,columns].values: ", group[1].loc[:,columns].values)#.tolist() )
      
        len_group = len( group[1].loc[:,columns_TODO].values)
        list_val = group[1].loc[:,columns_TODO].values
        if len_group > 1:
            for i in range(1, len_group):
                #print ("Comparing list_val[i]= " , list_val[i], " with list_val[i-1]=", list_val[i-1])
                (annot_1, annot_2) = compare_annotations(list_val[i], list_val[i-1])
                #print ("Identical values: ", list_val[i])            
            words.extend(annot_1)
            words.extend(annot_2)
    #print ("WORDS: ", words)
    return words

if __name__ == "__main__":
    appended_annots = []
    path = 'Attachments' # The file '78065D, 71869' removed because students made quite a few malformed annotations and did not understand the task
    for filename in os.listdir(path):
        full_filename = os.path.join(path, filename)        
        data = pd.read_excel(full_filename)
        if 'id' in data.columns: #Mandarin files do not have id's        
            appended_annots.append(data)
  
    all_frame = pd.concat(appended_annots) # all annotations into one frame
    columns = [['taste words'], ['overall words'], ['environment words'], ['service words'], ['sanitation words'], ['worthy words']] 
    for column_name in columns:
        positive = set()
        negative = set()
        neutral = set()
        taste_words = parse_frame(all_frame, column_name)
        for word in taste_words:
            if '+' in word:
                positive.add(word)
            elif '-' in word:
                negative.add(word)
            elif '0' in word:
                neutral.add(word)
        print ("COLUMN: ", column_name)
        print ("POSITIVE: ", positive)
        print ("NEGATIVE: ", negative)
        print ("NEUTRAL: ", neutral)






OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  小籠包:皮薄餡靚+/包住湯汁+
OCCURENCE DEBUG:  素菜筍尖炒年糕:味道幾濃+/唔錯+
OCCURENCE DEBUG:  小籠包:皮薄餡靚+/素菜筍尖炒年糕:味道濃+
OCCURENCE DEBUG:  平時食開既唔同0
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  炆雞:好入味+
OCCURENCE DEBUG:  小籠包:好好味+/嫩嫩地+
OCCURENCE DEBUG:  皮薄+/肉汁多+
OCCURENCE DEBUG:  炆雞:好入味+/好好味+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  食物:無以前咁好食-
OCCURENCE DEBUG:  味道可以0
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  烏冬:有口感+
OCCURENCE DEBUG:  烏冬:有口感+
OCCURENCE DEBUG:  皮:厚薄適中+
OCCURENCE DEBUG:  餡:多+/貼住層皮0
OCCURENCE DEBUG:  羊肉:唔太羶+
OCCURENCE DEBUG:  肉汁:唔會過多+/唔會太鹹+
OCCURENCE DEBUG:  皮:厚薄適中+
OCCURENCE DEBUG:  餡:多+/貼住層皮0
OCCURENCE DEBUG:  羊肉:唔太羶+
OCCURENCE DEBUG:  肉汁:唔會過多+/唔會太鹹+
OCCURENCE DEBUG:  肉汁:好多好鹹-
OCCURENCE DE

OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  肉醬意粉:十分好+
OCCURENCE DEBUG:  肉醬意粉:十分好+
OCCURENCE DEBUG:  來的時候像黑炭頭.但裡面只有三成熟-,再有招牌.都是極之淡而無味-
OCCURENCE DEBUG:  來的時候像黑炭頭.但裡面只有三成熟-
OCCURENCE DEBUG:  再有招牌.都是極之淡而無味-
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  雞扒意粉:非常一般-/同農場差唔多.可能仲差少少-
OCCURENCE DEBUG:  雞肉:乾-/鞋鞋地-
OCCURENCE DEBUG:  汁:怪怪地-
OCCURENCE DEBUG:  雞扒意粉:非常一般-/同農場差唔多.可能仲差少少-
OCCURENCE DEBUG:  雞肉:乾-/鞋鞋地-
OCCURENCE DEBUG:  汁:怪怪地-
OCCURENCE DEBUG:  雞扒意粉：非常一般-/雞肉：幾乾又鞋鞋地-/個汁:味又怪怪地-
OCCURENCE DEBUG:  雞扒意粉:非常一般-/同農場差唔多.可能仲差少少-
OCCURENCE DEBUG:  雞肉:乾-/鞋鞋地-
OCCURENCE DEBUG:  汁:怪怪地-
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  牛肉小鍋米線:香香辣辣好好味+
OCCURENCE DEBUG:  牛肉：香香辣辣+/好好味+
OCCURENCE DEBUG:  濃濃湯底:正+/米線:好滑+/唔會太諗-/雲吞皮:厚咗少少-/味道都唔錯+
OCCURENCE DEBUG:  濃濃湯底:正+
OCCURENCE DEBUG:  米線:好滑+/唔會太諗+
OCCURENCE DEBUG:  雲吞皮:厚咗少少-/味道都唔錯+
OCCURENCE DEBUG:  豬軟骨：唔知係唔係好好食0
OCCURENCE DEBUG:  酸酸辣辣好正:+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  中辣好辣-/米線同湯底:麻麻地-/凍檸茶:太多冰-
OCCUREN

OCCURENCE DEBUG:  漢堡:鹹甜交集又幾夾+
OCCURENCE DEBUG:  鹹甜交集又幾夾0
OCCURENCE DEBUG:  手打牛肉:香脆+
OCCURENCE DEBUG:  手打牛肉:厚身之餘都好+
OCCURENCE DEBUG:  魚肉:嫩滑+/酒香:淡淡而發都值得加分+/朱古力:幾濃郁+/好飲+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  魚肉:更為嫩滑+/酒香淡淡+/朱古力:幾濃+
OCCURENCE DEBUG:  魚肉:嫩滑+/酒香:淡淡而發都值得加分+/朱古力:幾濃郁+/好飲+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  芝麻炸豬頸肉:脆卜卜+/.好過口癮+
OCCURENCE DEBUG:  芝麻炸豬頸肉:脆卜卜+/好過口癮+
OCCURENCE DEBUG:  芝麻炸豬頸肉:脆卜卜+/好過口癮+
OCCURENCE DEBUG:  芝麻炸豬頸肉:脆卜卜+/.好過口癮+
OCCURENCE DEBUG:  豆腐沙律:非常辣+
OCCURENCE DEBUG:  豆腐沙律:非常辣同埋佢-
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  雲吞:略咸-/好香松露味+/失色-
OCCURENCE DEBUG:  略咸-
OCCURENCE DEBUG:  容易入口+
OCCURENCE DEBUG:  入口冇苦味+
OCCURENCE DEBUG:  好重果香+
OCCURENCE DEBUG:  正+
OCCURENCE DEBUG:  生蠔:幾生鮮+/不錯喔+
OCCURENCE DEBUG:  生蠔：新鮮+；容易入口+/入口冇苦味+/好重果香+
OCCURENCE DEBUG:  生牛肉:幾鮮嫩+
OCCURENCE DEBUG:  生牛肉:幾鮮嫩嘅+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE D

OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  就腳+/唔錯+
OCCURENCE DEBUG:  就腳+/唔錯+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  我一個女仔覺得都未夠食-
OCCURENCE DEBUG:  味道:唔錯+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  我一個女仔覺得都未夠食-
OCCURENCE DEBUG:  味道:唔錯+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  都要排隊的-
OCCURENCE DEBUG:  拉花咖啡:出名+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  味精太勁了-
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  可以任飲.很多選擇+
OCCURENCE DEBUG:  很多選擇+
OCCURENCE DEBUG:  nan
OC

OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  朱古力粒粒雞蛋仔:推薦+
OCCURENCE DEBUG:  朱古力粒粒雞蛋仔:推薦+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  難俾好評0
OCCURENCE DEBUG:  難啲俾好評-
OCCURENCE DEBUG:  鍾意+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  我地既最愛既路邊煱+
OCCURENCE DEBUG:  
OCCURENCE DEBUG:  最愛+
OCCURENCE DEBUG:  路邊煱:我地既最愛+
OCCURENCE DEBUG:  我地既最愛既路邊煱+
OCCURENCE DEBUG:  
OCCURENCE DEBUG:  呢個雜菌花甲蟹鍋入面有大肉蟹.花甲.雜菌+/未食都已經被賣相吸引左+
OCCURENCE DEBUG:  賣相:吸引+
OCCURENCE DEBUG:  賣相:吸引+
OCCURENCE DEBUG:  呢個雜菌花甲蟹鍋入面有大肉蟹.花甲.雜菌+/未食都已經被賣相吸引左+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  賣相:贏晒+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  物有所值:+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  物有所值:+
OCCURENCE DEBUG:  nan
OCCUR

OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  環境:幾好+
OCCURENCE DEBUG:  環境:幾好+
OCCURENCE DEBUG:  幾好
OCCURENCE DEBUG:  環境:幾好+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  坐得幾辛苦下-/太迫了-
OCCURENCE DEBUG:  坐得幾辛苦下-/太迫-
OCCURENCE DEBUG:  地下嘅:位置都幾好+
OCCURENCE DEBUG:  行過去:好快+
OCCURENCE DEBUG:  nan
OCCURENCE DEBUG:  

ValueError: too many values to unpack (expected 2)